In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
# with open('mismatching_reactions_transcription_only.txt', 'r') as infile:
#     juans = pandas.read_csv(infile, sep = '\t', dtype = str, index_col = 0)
# juans = juans[juans.index.str.contains('from') & (juans['iJL1673'] == '0.0')]
# index = [ x[14:] for x in juans.index ]
# index

In [4]:
with open('/opt/git-repositories/ecolime.SBRG/ecolime/building_data/TUs_from_ecocyc.txt', 'r') as infile:
    original = pandas.read_csv(infile, sep = '\t', keep_default_na = False)
original

TU_id = [ x for x in original['TU_id']]
starts = [ int(x) for x in original['start']]
stops = [ int(x) for x in original['stop']]

In [5]:
original

,TU_id,start,stop,tss,strand,rho_dependent,sigma
0,TU0_13350_from_RpoD_mono,1700257,1701258,None,+,True,RpoD_mono
1,TU0_12877_from_RpoD_mono,303077,303406,None,-,True,RpoD_mono
2,TU0_7622_from_RpoD_mono,542485,545635,545635,-,True,RpoD_mono
3,TU00158_from_RpoD_mono,1165215,1166612,1165215,+,True,RpoD_mono
4,TU0_8340_from_EG11355_MONOMER,2010724,2011117,2011117,-,True,FliA_mono
...,...,...,...,...,...,...,...
3509,TU0_13407_from_RpoD_mono,1877427,1878018,1878018,-,True,RpoD_mono
3510,TU0_13145_from_RpoD_mono,1193050,1194174,None,-,True,RpoD_mono
3511,TU0_12807_from_RpoD_mono,84188,85312,84188,+,True,RpoD_mono
3512,TU0_13883_from_RpoD_mono,3320755,3322933,None,-,True,RpoD_mono


In [6]:
from Bio import SeqIO
contigs = []
for contig in SeqIO.parse('/opt/git-repositories/ecolime.SBRG/ecolime/building_data/NC_000913.2.gb', 'genbank'):
    contigs.append(contig)

dct = {}
for feature in contigs[0].features:
    for tu_id, start, stop in zip(TU_id, starts, stops):
        if feature.type in ['CDS', 'rRNA', 'tRNA', 'tmRNA', 'ncRNA'] and feature.location.start >= start-2 and feature.location.end <= stop:
            dct.setdefault((tu_id, start, stop), set()).add(feature.qualifiers.get('locus_tag', ['None'])[0])
            if feature.qualifiers.get('locus_tag', False) == False:
                print(feature.__dict__)

original['replicon'] = 'NC_000913.2'
original['genes'] = original.apply(lambda x: ','.join(list(dct.get((x['TU_id'], int(x['start']), int(x['stop'])), ['']))), axis = 1)
original['genes'] = original['genes'].apply(lambda x: ','.join(sorted(x.split(','))))
original.to_excel('TUs_from_ecolime_plus_genes.xlsx', index = False)
original.drop_duplicates().to_csv('TUs_from_ecolime_plus_genes.txt', sep = '\t', index = False)

In [7]:
dct[('TU00041_from_RPOH_MONOMER', 4212256, 4213232)]

{'b4013'}

In [8]:
incorrect_sigma_32 = [
    "TU0_3463_from_RPOH_MONOMER",
    "TU0_1181_from_RPOH_MONOMER",
    "TU0_1182_with_TERM0_1116_from_RPOH_MONOMER",
    "TU0_1182_with_TERM0_1115_from_RPOH_MONOMER",
    "TU0_1183_with_TERM0_1117_from_RPOH_MONOMER",
    "TU0_1186_from_RPOH_MONOMER",
    "TU0_1187_from_RPOH_MONOMER",
    "TU0_1189_from_RPOH_MONOMER",
    "TU0_1191_from_RPOH_MONOMER"
    ]

dct = { k:f'#{k}' for k in incorrect_sigma_32 }

with open('TUs_from_ecolime_plus_genes.txt', 'r') as infile:
    new = pandas.read_csv(infile, sep = '\t', keep_default_na = False)
    
new['genes'] = new['genes'].apply(lambda x: ','.join(sorted(x.split(','))))
new['TU_id'] = new['TU_id'].replace(dct)
new.columns = ['TU_id', 'start', 'stop', 'tss', 'strand', 'rho_dependent', 'rnapol', 'replicon', 'genes']
cols = ['TU_id', 'replicon', 'genes', 'start', 'stop', 'tss', 'strand', 'rho_dependent', 'rnapol']
new.drop_duplicates().sort_values(['start', 'TU_id'])[cols].to_csv('TUs_from_ecolime_plus_genes.txt', sep = '\t', index = False)